In [ ]:
import os
import numpy as np

from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
PROJECT_DIR = '/content/drive/MyDrive/Colab Notebooks/teachable-gradio'
PROJECT_NAME = 'smp5_hanaca'
TEACHABLE_MODEL_DIR = f'{PROJECT_DIR}/{PROJECT_NAME}/converted_keras'
TEACHABLE_TRAIN_DIR = f'{PROJECT_DIR}/{PROJECT_NAME}/train'
TEACHABLE_TEST_DIR = f'{PROJECT_DIR}/{PROJECT_NAME}/test'
SHAPE = (224, 224)

In [ ]:
# load the model and label
model = load_model(f"{TEACHABLE_MODEL_DIR}/keras_model.h5", compile=False)
class_names = open(f"{TEACHABLE_MODEL_DIR}/labels.txt", "r").readlines()
class_names = [cn.replace('\n', '') for cn in class_names]


def pre_process_image(image):
    # resizing the image to be at least 224x224 and then cropping from the center
    image = ImageOps.fit(image, SHAPE, Image.LANCZOS)

    # turn the image into a numpy array
    image_array = np.asarray(image)

    # normalize the image
    image_array = (image_array.astype(np.float32) / 127.5) - 1

    # reshape the image to batch format
    image_array = image_array.reshape((-1, SHAPE[0], SHAPE[1], 3))

    return image_array


def classify_image(image):
    image = pre_process_image(image)
    prediction = model.predict(image)
    confidences = {class_names[i]: float(prediction[0][i]) for i in range(len(class_names))}
    return confidences

In [ ]:
def evaluate_model_on_dataset(dataset_dir):
    true_labels, predicted_labels, class_names_readed = predict_dataset_dir(dataset_dir)
    cm = confusion_matrix(true_labels, predicted_labels)
    report = classification_report(true_labels, predicted_labels, target_names=class_names_readed)
    return cm, report, class_names_readed

def predict_dataset_dir(dataset_dir):
    # initiate empty list
    true_labels = []
    predicted_labels = []
    class_names_readed = []

    # get and sort folders
    listdir = np.array(os.listdir(dataset_dir))
    class_names_index = [int(cn.split(' ')[0]) for cn in listdir]
    idx = np.argsort(class_names_index)
    listdir = listdir[idx]

    # test images on folders
    for class_name in listdir:
        class_dir = os.path.join(dataset_dir, class_name)
        for image_name in os.listdir(class_dir)[:2]:
            try:
              image_path = os.path.join(class_dir, image_name)
              image = Image.open(image_path)
              true_class_idx = class_names.index(class_name)
              image = pre_process_image(image)
            except:
              continue
            prediction = model.predict(image, verbose=0)
            predicted_class_idx = np.argmax(prediction)

            true_labels.append(true_class_idx)
            predicted_labels.append(predicted_class_idx)
            if class_name not in class_names_readed:
              class_names_readed.append(class_name)

    # fix numbering start from zero
    true_labels = np.array(true_labels) - 1
    predicted_labels = np.array(predicted_labels) - 1
    return true_labels, predicted_labels, class_names_readed


# Evaluate on the training set
train_cm, train_report, train_class_names_readed = evaluate_model_on_dataset(TEACHABLE_TRAIN_DIR)
print("Training Set Confusion Matrix:")
print(train_cm)
print("Training Set Classification Report:")
print(train_report)

# # Evaluate on the validation/test set
# val_cm, val_report, val_class_names_readed = evaluate_model_on_dataset(TEACHABLE_TEST_DIR)
# print("Validation Set Confusion Matrix:")
# print(val_cm)
# print("Validation Set Classification Report:")
# print(val_report)

In [ ]:
cm

In [ ]:
report

In [ ]:
len(class_names_readed)

In [ ]:
len(class_names_readed)

In [ ]:
true_labels